In [2]:
import mysql.connector as mysql
import json

with open('config.json', 'r') as read_file:
    client = json.load(read_file)

In [220]:
db = mysql.connect(
    host = 'localhost',
    buffered = True,
    user = client['MySQL User'],
    passwd = client['MySQL Pass'],
    database = 'scopus'
)

# column names
subject_col = ['asjc_code', 'top', 'middle', 'low']
source_col = ['source_id_scp', 'title', 'url', 'type', 'issn', 'e_issn', 'isbn', 'publisher', 'country']
source_subject_col = ['source_id', 'subject_id']
paper_col = ['paper_id_scp', 'eid', 'title', 'type', 'type_description', 'abstract', 'total_author', 'open_access', 'cited_cnt', 'url', 'article_no', 'fund_no', 'retrieval_time', 'source_id', 'doi', 'volume', 'issue', 'date', 'page_range']
author_col = ['author_id_scp', 'first', 'last', 'initials', 'sex', 'type', 'rank', 'email',]
paper_author_col = ['paper_id', 'author_id', 'author_no']
author_profile_col = ['author_id', 'url', 'type']
department_col = ['department_id', 'name', 'abbreviation', 'type', 'lat', 'lng']
author_department_col = ['author_id', 'department_id']
institution_col = ['institution_id_scp', 'name', 'abbreviation', 'city', 'country', 'url', 'type', 'lat', 'lng']
department_institution_col = ['department_id', 'institution_id']

subject_q = '''INSERT INTO subject (asjc_code, top, middle, low) VALUES (%s, %s, %s, %s)'''
source_q = '''INSERT INTO source (source_id_scp, title, url, type, issn, e_issn, isbn, publisher, country) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'''
source_subject_q = '''INSERT INTO source_subject (source_id, subject_id) VALUES (%s, %s)'''
paper_q = '''INSERT INTO paper (paper_id_scp, eid, title, type, type_description, abstract, total_author, open_access, cited_cnt, url, article_no, fund_no, retrieval_time, source_id, doi, volume, issue, date, page_range) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''
author_q = '''INSERT INTO author (author_id_scp, first, last, initials, sex, type, rank, email) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'''
paper_author_q = '''INSERT INTO paper_author (paper_id, author_id, author_no) VALUES (%s, %s, %s)'''
author_profile_q = '''INSERT INTO author_profile (author_id, url, type) VALUES (%s, %s, %s)'''
department_q = '''INSERT INTO department (department_id, name, abbreviation, type, lat, lng) VALUES (%s, %s, %s, %s, %s, %s)'''
author_department_q = '''INSERT INTO author_department (author_id, department_id) VALUES (%s, %s)'''
institution_q = '''INSERT INTO institution (institution_id_scp, name, abbreviation, city, country, url, type, lat, lng) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'''
department_institution_q = '''INSERT INTO department_institution (department_id, institution_id) VALUES (%s, %s)'''

cols = [
    subject_col, source_col, source_subject_col, paper_col, author_col, 
    paper_author_col, author_profile_col, department_col, author_department_col, 
    institution_col, department_institution_col
]

col_names = [
    'subject', 'source', 'source_subject', 'paper', 'author', 
    'paper_author', 'author_profile', 'department', 'author_department', 
    'institution', 'department_institution'
]

for name, col, que in zip(col_names, cols, ques):
    table_name = name
    table_col = col
    q = f'INSERT INTO {table_name} ({", ".join(table_col)}) VALUES ({"%s, " * (len(table_col) - 1)}%s)'

print(db)
cursor = db.cursor()
db.close()

In [2]:
# modules

import os
import io
import csv
import json
from collections import OrderedDict
from datetime import datetime
import random

path = 'data\\Sharif University of Technology'
files = list(os.walk(path))[0][2]

with io.open(os.path.join(path, files[0]), 'r', encoding='utf8') as raw:
    data = json.load(raw)

faculties = []
with io.open('data\\faculties.csv', 'r', encoding='utf-8-sig') as csvFile:
    reader = csv.DictReader(csvFile)
    for row in reader:
        if row['Scopus']:
            row['Scopus'] = list(map(int, row['Scopus'].split(',')))
        faculties.append(row)

asjc = []
with io.open('data\\ASJC Codes.csv', 'r', encoding='utf-8-sig') as csvFile:
    reader = csv.DictReader(csvFile)
    for row in reader:
        asjc.append(row)

sources = []
with io.open('data\\Scopus Sources.csv', 'r', encoding='utf-8-sig') as csvFile:
    reader = csv.DictReader(csvFile)
    for row in reader:
        row.pop('Active', None)
        row.pop('Discontinued', None)
        row.pop('Coverage', None)
        row.pop('2016 CiteScore', None)
        row.pop('2017 CiteScore', None)
        row.pop('2018 CiteScore', None)
        row.pop('Medline-sourced', None)
        row.pop('Open Access', None)
        row.pop('Articles in Press Included', None)
        row.pop('Added to list April 2019', None)
        row.pop('Title history indication', None)
        row.pop('Related title to title history indication', None)
        row.pop('Other related title 1', None)
        row.pop('Other related title 2', None)
        row.pop('Other related title 3', None)
        row.pop('Publisher imprints grouped to main Publisher', None)
        
        row['ASJC'] = [int(code) for code in row['ASJC'].split(';') if code != '']
        sources.append(row)

In [ ]:
for file in files:
#     print(file)
    year = file.split('.')[0].split('_')[-4][1:]
    with io.open(os.path.join(path, file), 'r', encoding='utf8') as raw:
        data = json.load(raw)
    data = data['search-results']['entry']
    for paper in data:
        print(int(paper['dc:identifier'].split(':')[1]))
        rnd_source = random.randint(100000,200000)
        source_info = [
            (int(paper['source-id']) if 'source-id' in paper.keys() else rnd_source),
            (paper['prism:publicationName'] if 'prism:publicationName' in paper.keys() else 'No Name!'),
            'https://www.scopus.com/sourceid/', # url
            (paper['prism:issn'] if 'prism:issn' in paper.keys() else None),
            (paper['prism:isbn'][0]['$'] if 'prism:isbn' in paper.keys() else None),
            None, # **subject
            None, # **publisher
            (paper['prism:aggregationType'] if 'prism:aggregationType' in paper.keys() else None),
        ]
        cursor.execute(source_q, source_info)
        
        paper_info = [
            int(paper['dc:identifier'].split(':')[1]),
            paper['eid'],
            paper['dc:title'],
            (paper['subtype'] if 'subtype' in paper.keys() else None),
            (paper['subtypeDescription'] if 'subtypeDescription' in paper.keys() else None),
            (paper['dc:description'] if 'dc:description' in paper.keys() else None),
            paper['author-count']['$'],
            (paper['openaccess'] if 'openaccess' in paper.keys() else None),
            paper['citedby-count'],
            paper['link'][-2]['@href'],
            (paper['article-number'] if 'article-number' in paper.keys() else None),
            (paper['fund-no'] if 'fund-no' in paper.keys() else None),
            datetime.utcfromtimestamp(int(file.split('.')[0].split('_')[-1])).strftime('%Y-%m-%d %H:%M:%S'),
            datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            (int(paper['source-id']) if 'source-id' in paper.keys() else rnd_source),
            (paper['prism:doi'] if 'prism:doi' in paper.keys() else None),
            (paper['prism:volume'] if 'prism:volume' in paper.keys() else None),
            (paper['prism:issueIdentifier'] if 'prism:issueIdentifier' in paper.keys() else None),
            (datetime.strptime(paper['prism:coverDate'], '%Y-%m-%d').strftime('%Y-%m-%d') if 'prism:coverDate' in paper.keys() else year),
            (paper['prism:pageRange'] if 'prism:pageRange' in paper.keys() else None),
        ]
#         cursor.execute(paper_q, paper_info)
        
        author_info = []
        paper_author_info = []
        for auth in paper['author']:
            author_info.append(
                [
                    int(auth['authid']),
                    auth['given-name'],
                    auth['surname'],
                    auth['initials'],
                    auth['author-url'],
                    None, # sex
                    None, # type
                    None, # rank
                    None, # email
                ]
            )
            
            paper_author_info.append(
                [
                    paper_info[0],
                    int(auth['authid']),
                    auth['@seq'],
                ]
            )
        db.commit()
db.close()
print('Finished!')

In [159]:
class Database:
    
    table_ids = {
        'source': {'order': 0, 'id': ['source_id_scp']},
        'subject': {'order': 0, 'id': ['asjc_code']},
        'country': {'order': 0, 'id': ['country_id']},
        'paper_funding': {'order': 0, 'id': ['agency_id_scp']},
        
        'source_subject': {'order': 1, 'id': ['source_id', 'subject_id']},
        'paper': {'order': 1, 'id': ['paper_id_scp']},
        
        'author': {'order': 2, 'id': ['author_id_scp']},
        'keyword': {'order': 2, 'id': ['keyword_id']},
        
        'paper_author': {'order': 3, 'id': ['paper_id', 'author_id']},
        'paper_keyword': {'order': 3, 'id': ['paper_id', 'keyword_id']},
        'department': {'order': 3, 'id': ['department_id']},
        'author_profile': {'order': 3, 'id': ['profile_id', 'author_id']},
        
        'author_department': {'order': 4, 'id': ['author_id', 'department_id']},
        'institution': {'order': 4, 'id': ['institution_id_scp']},
        
        'department_institution': {'order': 5, 'id': ['department_id', 'institution_id']},
    }
    
    def __init__(self, config: dict, db_name: str, host: str = 'localhost', port: int = 3306, buffered: bool = True):
        print('@ __init__')
        self._params = {
            'host': host,
            'buffered': buffered,
            'user': config['MySQL User'],
            'pass': config['MySQL Pass'],
        }
        self.db_name = db_name
        self.db = None
        self.cursor = None
        self.tables = []
        print('__init__ done!')
    
    def _connect(self):
        print('@ _connect')
        if not self.db:
            self.db = mysql.connect(
                host = self._params['host'],
                buffered = self._params['buffered'],
                user = self._params['user'],
                password = self._params['pass'],
                database = self.db_name
            )
        print('_connect done!')
        return self.db
    
    def _cursor(self):
        print('@ _cursor')
        if not self.db:
            self._connect()
        if not self.db.is_connected():
            self.db.reconnect()
        if not self.cursor:
            self.cursor = self.db.cursor()
        print('_cursor done!')
        return self.cursor
    
    def _execute(self, query, values = [], fetch: bool = False, many: bool = False, close_cursor: bool = False):
        print('@ _execute')
        if many:
            self._cursor().executemany(query, values)
        else:
            self._cursor().execute(query, values)
        if fetch:
            server_response = self.cursor.fetchall()
        else:
            server_response = self.cursor
        print('_execute done!')
        if close_cursor:
            self.cursor.close()
            self.cursor = None
        return server_response
    
    def _close(self):
        print('@ _close')
        if self.db.is_connected():
            self.db.close()
        print('Closed!')
    
    def _show_tables(self):
        print('@ _show_tables')
        return [table[0] for table in self._execute(query = 'SHOW TABLES', fetch = True)]
    
    def _has_table(self, table_name):
        print('@ _has_table')
        table_names = self._show_tables()
        if table_name in table_names:
            print('_has_table done!')
            return True
        print('_has_table done!')
        return False
    
    def _column_names(self, table_name):
        print('@ _column_names')
        return [col[0] for col in self.describe(table_name)]
    
    def describe(self, table_name: str = ''):
        print('@ describe')
        if table_name:
            query = f'DESCRIBE {table_name}'
            print('describe done!')
            return self._execute(query = query, fetch = True)
        server_response = self._show_tables()
        for table in server_response:
            self.tables.append({table: self.describe(table)})
        print('describe done!')
        return self.tables
    
    def _read(self, table_name: str, search: dict, select = '*', result_columns: bool = False):
        print('@ _read')
        if not self._has_table(table_name):
            return f'Error! "{table_name}" table not found'
        
        if search:
            query = (f'SELECT {select} FROM {table_name} WHERE ' 
                + ' AND '.join([f'{k} {v["operator"]} {v["value"]}' for k, v in search.items()]))
        else:
            query = f'SELECT {select} FROM {table_name}'
        server_response = self._execute(query = query, fetch = True, close_cursor = True)
        print('got the response from _execute')
        if result_columns:
            result = []
            if select == '*':
                column_names = self._column_names(table_name)
            else:
                column_names = [column.strip() for column in select.split(',')]
            for row in server_response:
                result.append({name: value for name, value in zip(column_names, row)})
            print('_read done!')
            return result
        print('_read done!')
        return server_response
    
    # def has_row(self, table_name, row_ids: dict):
    #     print('@ has_row')
    #     search = {}
    #     for row_id in row_ids:
    #         search[row_id] = {'value': row_ids[row_id], 'operator': '='}
    #     server_response = self._read(table_name, search, select = 'COUNT(*)')
    #     print('has_row done!')
    #     return server_response[0][0]
    
    def _insert_one(self, table_name, data: dict):
        # data is a list of dictionaries
        print('@ _insert_one')
        if not self._has_table(table_name):
            return f'Error! "{table_name}" table not found'
        
        table_columns = self._column_names(table_name)
        data_columns = list(data.keys())
        for col in data_columns:
            if col not in table_columns:
                return f'Error! "{col}" column not found'
        
        query = f'INSERT INTO {table_name} ({", ".join(data_columns)}) VALUES ({"%s, " * (len(data_columns) - 1)}%s)'
        
        # check if the record already exists
        id_columns = Database.table_ids[table_name]['id']
        search = {id_column: {'value': data[id_column], 'operator': '='} for id_column in id_columns}
        print(search)
        server_response = self._read(table_name, search)
        if server_response: # record exists, let's return the its primary key
            server_response = server_response[-1][0]
            return {'msg': f'Table "{table_name}" already has this record', 'value': server_response}
        
        # record is new
        values = tuple(data[col] for col in data_columns)
        try:
            self._execute(query, values)
            print('_insert_one done!')
            self.db.commit()
            last_id = self.cursor.lastrowid
            self._close()
            return {'msg': f'Record added to "{table_name}"', 'value': last_id}
        except Exception as e:
            print(f'error here: {e}')
            self._close()
        
    def _insert_many(self, table_name, data: list):
        # data is a list of dictionaries
        print('@ _insert')
        if not self._has_table(table_name):
            return f'Error! "{table_name}" table not found'
        
        # assuming all data rows have the same columns
        # data is a list of dictionaries, of which the keys are column names
        table_columns = self._column_names(table_name)
        data_columns = list(data[0].keys())
        for col in data_columns:
            if col not in table_columns:
                return f'Error! "{col}" column not found'
        
        query = f'INSERT INTO {table_name} ({", ".join(data_columns)}) VALUES ({"%s, " * (len(data_columns) - 1)}%s)'
        values = []
        id_columns = Database.table_ids[table_name]['id']
        skipped_rows = 0
        total_rows = len(data)
        for row in data:
            search = {id_column: {'value': row[id_column], 'operator': '='} for id_column in id_columns}
            if self._read(table_name, search):
                skipped_rows += 1
                continue
            values.append(tuple(row[col] for col in data_columns))
        try:
            self._execute(query, values, many = True)
            print('_insert done!')
            self.db.commit()
            last_id = self.cursor.lastrowid
            self._close()
            return {'msg': f'{total_rows - skipped_rows} records added ({skipped_rows} already existed)', 'value': last_id}
        except Exception as e:
            print(f'error here: {e}')
            self._close()
    
    def raw_insert(self, data: list, retrieval_time):
        print('@ raw_insert')
        # data is a dictionary containing the info about 1 paper
        warnings = data_inspector(data)
        if 'openaccess' in warnings:
            data['openaccess'] = '0'
            warnings.pop('openaccess')
        if 'author:afid' in warnings:
            warnings.pop('author:afid')
        if len(warnings):
            return {'warnings': warnings}
        
        keys = data.keys()
        
        paper_url = ''
        for link in data['link']:
            if link['@ref'] == 'scopus':
                paper_url = link['@href']
                break

        paper_id_scp = int(data['dc:identifier'].split(':')[1])
        if self._read('paper', {'paper_id_scp': {'value': paper_id_scp, 'operator': '='}}):
            return {'warnings': ['paper exists']}
        
        source_id_scp = int(data['source-id'])
        agency_id_scp = key_get(data, keys, 'fund-no')
        if agency_id_scp == 'undefined':
            agency_id_scp = None
        
        source_info = {
            'source_id_scp': source_id_scp, 
            'title': data['prism:publicationName'], 
            'url': f'https://www.scopus.com/sourceid/{source_id_scp}', 
            'type': key_get(data, keys, 'prism:aggregationType'), 
            'issn': key_get(data, keys, 'prism:issn'), 
            'e_issn': key_get(data, keys, 'prism:eIssn'), 
            'isbn': key_get(data, keys, 'prism:isbn'), 
            'publisher': None, 
            'country_id': None
        }
        
        source_id = self._insert_one('source', source_info)['value']
        
        agency_id = None
        if agency_id_scp:
            paper_funding_info = {
                'agency_id_scp': key_get(data, keys, 'fund-no'), 
                'agency':  key_get(data, keys, 'fund-sponsor'), 
                'agency_acronym': key_get(data, keys, 'fund-acr'), 
            }
            agency_id = self._insert_one('paper_funding', paper_funding_info)['value']            
            
        paper_info = {
            'paper_id_scp': paper_id_scp,
            'eid': data['eid'],
            'title': data['dc:title'],
            'type': data['subtype'],
            'type_description': key_get(data, keys, 'subtypeDescription'),
            'abstract': key_get(data, keys, 'dc:description'),
            'total_author': key_get(data, keys, 'author-count'),
            'open_access': data['openaccess'],
            'cited_cnt': data['citedby-count'],
            'url': paper_url,
            'article_no': key_get(data, keys, 'prism:volume'),
            'agency_id': agency_id,
            'retrieval_time': retrieval_time,
            'source_id': source_id,
            'doi': key_get(data, keys, 'prism:doi'),
            'volume': key_get(data, keys, 'prism:volume'),
            'issue': key_get(data, keys, 'prism:issueIdentifier'),
            'page_range': key_get(data, keys, 'prism:pageRange'),
            'date': data['prism:coverDate'],
        }
        
        paper_id = self._insert_one('paper', paper_info)['value']
        print('raw_insert done')
        return [source_id, agency_id, paper_id]
#             'author': {},
#             'keyword': {},
#             'paper_author': {},
#             'paper_keyword': {},
#             'author_profile': {},
#             'department': {},
#             'author_department': {},
#             'institution': {},
#             'department_institution': {},
#         }

#         author_info = []
#         paper_author_info = []
#         for author in data['author']:
#             keys = author.keys()
#             author_id = int(author['authid'])
#             author_info.append(
#                 {
#                     'author_id_scp': author_id,
#                     'first': key_get(author, keys, 'given-name'),
#                     'last': key_get(author, keys, 'surname'),
#                     'initials' key_get(author, keys, 'initials'),
#                     # f'https://www.scopus.com/authid/detail.uri?authorId={auth["authid"]},
#                 }
#             )
#             paper_author_info.append(
#                 {
#                     'paper_id': None, # paper_id
#                     'author_id': None, # author_id
#                     'author_no': int(author['@seq']),
#                 }
#             )

In [130]:
def data_inspector(data: dict):
    warnings = []
    top_keys = [
        'source-id', 'prism:publicationName', 'prism:coverDate',
        'dc:identifier', 'eid', 'dc:title', 'subtype', 'author-count', 'openaccess', 'citedby-count', 'link', 
        'author', 'affiliation',
    ]
    author_keys = ['authid', '@seq', 'afid']
    affiliation_keys = ['afid', 'affilname']
    
    keys = data.keys()
    for key in top_keys:
        if key not in keys:
            warnings.append(key)
    if 'link' not in warnings:
        if all(link['@ref'] != 'scopus' for link in data['link']):
            warnings.append('paper url')
    if 'author' not in warnings:
        for author in data['author']:
            keys = author.keys()
            for key in author_keys:
                if key not in keys:
                    warnings.append(f'author:{key}')
    if 'affiliation' not in warnings:
        for affiliation in data['affiliation']:
            keys = affiliation.keys()
            for key in affiliation_keys:
                if key not in keys:
                    warnings.append(f'affiliation:{key}')
    return warnings

def key_get(data: dict, keys, key: str):
    result = (data[key] if key in keys else None)
    if type(result) == list:
        return result[0]['$']
    if type(result) == dict:
        return result['$']
    return result

In [131]:
d = Database(config = client, db_name = 'scopus')
d._insert_many('paper_funding', [{'agency_id_scp': 27, 'agency': 'FQatar', 'agency_acronym': 'QNSF'}])#,{'agency_id_scp': 13, 'agency': 'US', 'agency_acronym': 'NSF'}])
# d._insert_many('source_subject', [{'source_id': 2, 'subject_id': 2},{'subject_id': 3, 'source_id': 1}])
# d._read('paper_funding', {'agency_id_scp': {'value': 20, 'operator': '='}}, result_columns=True)
# d._insert_one('paper_funding', {'agency_id_scp': 21, 'agency': 'FQatar', 'agency_acronym': 'QNSF'})#,{'agency_id_scp': 13, 'agency': 'US', 'agency_acronym': 'NSF'}])
# d._read('source_subject', {'source_id': {'value': 1, 'operator': '='}, 'subject_id': {'value': 3, 'operator': '='}}, result_columns=True)
# d.has_row('paper_funding', [['agency_id',13]])
# d._table_order()
# print(d.describe('subject'))
d._close()

@ __init__
__init__ done!
@ _insert
@ _has_table
@ _show_tables
@ _execute
@ _cursor
@ _connect
_connect done!
_cursor done!
_execute done!
_has_table done!
@ _column_names
@ describe
describe done!
@ _execute
@ _cursor
_cursor done!
_execute done!
@ _read
@ _has_table
@ _show_tables
@ _execute
@ _cursor
_cursor done!
_execute done!
_has_table done!
@ _execute
@ _cursor
_cursor done!
_execute done!
got the response from _execute
_read done!
@ _execute
@ _cursor
_cursor done!
_execute done!
_insert done!
@ _close
Closed!
@ _close
Closed!


In [162]:
import os
import io
import csv
import json
from collections import OrderedDict
from datetime import datetime
import random

cnt = 0
db = Database(config = client, db_name = 'scopus')
path = 'data\\Sharif University of Technology'
files = list(os.walk(path))[0][2]
for file in files[:1]:
    with io.open(os.path.join(path, file), 'r', encoding='utf8') as raw:
        data = json.load(raw)
    data = data['search-results']['entry']
    ret_time = datetime.utcfromtimestamp(int(file.split('.')[0].split('_')[-1])).strftime('%Y-%m-%d %H:%M:%S')
    for paper in data[:1]:
        warnings = data_inspector(paper)
        ins_id = db.raw_insert(paper, retrieval_time=ret_time)
print(ins_id)

@ __init__
__init__ done!
@ raw_insert
@ _read
@ _has_table
@ _show_tables
@ _execute
@ _cursor
@ _connect
_connect done!
_cursor done!
_execute done!
_has_table done!
@ _execute
@ _cursor
_cursor done!
_execute done!
got the response from _execute
_read done!
@ _insert_one
@ _has_table
@ _show_tables
@ _execute
@ _cursor
_cursor done!
_execute done!
_has_table done!
@ _column_names
@ describe
describe done!
@ _execute
@ _cursor
_cursor done!
_execute done!
{'source_id_scp': {'value': 21100828963, 'operator': '='}}
@ _read
@ _has_table
@ _show_tables
@ _execute
@ _cursor
_cursor done!
_execute done!
_has_table done!
@ _execute
@ _cursor
_cursor done!
_execute done!
got the response from _execute
_read done!
@ _insert_one
@ _has_table
@ _show_tables
@ _execute
@ _cursor
_cursor done!
_execute done!
_has_table done!
@ _column_names
@ describe
describe done!
@ _execute
@ _cursor
_cursor done!
_execute done!
{'paper_id_scp': {'value': 85059543553, 'operator': '='}}
@ _read
@ _has_table
@ _